In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [9]:
df = pd.read_csv('../data/train/sentiment_analysis_trainingset.csv')

In [90]:
df = pd.read_csv('../data/valid/sentiment_analysis_validationset.csv')

In [80]:
df = pd.read_csv('../data/testa/sentiment_analysis_testa.csv')

In [14]:
df

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2
5,5,"""烤鸭使用的蘸酱很地道，觉得烤鸭好不好吃在很大程度上都取决于这个酱料是否正宗。个人感觉他们家...",-2,-2,-2,-2,1,-2,-2,0,...,-2,-2,1,-2,0,1,-2,-2,1,-2
6,6,"""在好友圈很红的原因很诡异.竟然基本都是我外地的朋友转发的.不远千里跑来厦门买回去吃.吃完各...",-2,-2,-1,-2,-2,-2,-2,-1,...,-2,-2,-1,-2,-1,1,1,1,1,-2
7,7,"""中午饭点人比较多需要排队～2点左右去吃的刚好可以用团购～也不用排队～就是平安夜店员说的都不...",-2,-2,-2,0,1,-2,-2,-2,...,-2,-2,-2,-1,-1,0,-2,1,1,1
8,8,"""跟朋友相约25号看港囧，首映当天排片率超高，为了提前订好票，选了文投国际影城，为了看电影我...",-2,1,-2,-2,-2,-2,-2,1,...,1,-2,-2,-2,-2,1,-2,-2,1,1
9,9,"""首先感谢大众点评再次抽中我免费品尝火锅，同时也感谢古蜀雒味三星堆主题火锅商家提供这个平台，...",-2,-2,1,-2,1,-2,-2,-2,...,1,1,1,1,0,-2,1,-2,0,1


In [11]:
df.shape

(105000, 22)

## 转换原始数据为全类别模式

In [83]:
col_names = df.columns[2:]
df_new = df[df.columns[:2]]
for col_name in col_names:
    df_one_new = pd.get_dummies(df[col_name], prefix=col_name)
    cols = df_one_new.columns
    cols = cols[::-1]
    df_one_new = df_one_new[cols]
    df_new = pd.concat([df_new, df_one_new], axis=1)

In [84]:
df_new

,id,content
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首..."
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一..."
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么..."
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中..."
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被..."
5,5,"""尽管韩国烤肉店在无锡已经有很多家了，但因为味道好吃再加上在无锡很有人气，依旧挡不住新店的开..."
6,6,"""店铺在乙烯生活二区的西北角，旁边是国旅的旗舰店，门口就是红绿灯，挺好找的。因为这是两磅一的..."
7,7,"""朋友聚会来滴，这个地方真心不错哇，又能撸串，又能唱歌，还能看现场歌手演唱。最主要是可以上台..."
8,8,"""超喜欢这家的面食，经常来吃。这家风格及产品都很像京城御面，有时会怀疑是不是同一家的。面条比..."
9,9,"""广西阳朔，前一天晚上吃的特色啤酒鱼，就看到了隔壁有家螺蛳粉，第二天过来吃。个人比较喜欢粉，..."


In [85]:
df_new.shape

(15000, 2)

In [70]:
# write out train dataset
df_new.to_csv(path_or_buf='./data/df_train.csv', header=True, index=False)

In [78]:
# write out valid dataset
df_new.to_csv(path_or_buf='./data/df_valid.csv', header=True, index=False)

In [86]:
# write out testa dataset
df_new.to_csv(path_or_buf='./data/df_testa.csv', header=True, index=False)

In [51]:
df_new['content'].iloc[0]

'"吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢了，这哪舍得吃啊。"'

In [56]:
df_new[df_new.columns[2:]].shape

(105000, 80)

In [87]:
df_new[:2]

,id,content
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首..."
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一..."


In [3]:
df_testa_pred = pd.read_csv('./data/df_testa_pred.csv')

In [4]:
df_testa_pred

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""我想说他们家的优惠活动好持久啊，我预售的时候买的券，前两天心血来潮去吃的活动还在继续\n首...",-2,-2,-2,-2,-2,-2,-2,1,...,-2,-2,-2,-2,-2,1,-2,-2,1,-2
1,1,"""终于开到心心念念的LAB loft。第一次来就随便点也一些～【香辣虾意面】蛮辣的，但其实一...",-2,-2,-2,-2,-2,1,-2,0,...,-2,-2,-2,-2,1,1,-2,1,1,1
2,2,"""地理位置好，交通方便，就在124车站对面交通方便，很好，我晚上7点多去买的了，已经没有什么...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,1,1,0,-2,-2,1,1
3,3,"""运气很好，抽中了大众点评的霸王餐。这家主题餐厅心仪已久了，种种原因一直未能成行，没想到抽中...",-2,-2,-2,-2,1,-2,-2,-2,...,1,-2,-2,-2,-2,1,1,-2,1,-2
4,4,"""幸运随点评团体验霸王餐，心情好~蜀九香刚进驻泉州不久，招牌大名气响，以至于刚到店门口的我被...",-2,-2,-2,-2,-2,-2,-2,-2,...,1,-2,0,-2,-1,0,-2,-2,1,-2
5,5,"""尽管韩国烤肉店在无锡已经有很多家了，但因为味道好吃再加上在无锡很有人气，依旧挡不住新店的开...",-2,-2,-2,-2,-2,-2,-2,1,...,-2,-2,-2,-2,1,1,-2,-2,1,1
6,6,"""店铺在乙烯生活二区的西北角，旁边是国旅的旗舰店，门口就是红绿灯，挺好找的。因为这是两磅一的...",-2,1,1,-2,1,-2,-2,-1,...,1,1,-2,-2,-2,0,1,1,1,-2
7,7,"""朋友聚会来滴，这个地方真心不错哇，又能撸串，又能唱歌，还能看现场歌手演唱。最主要是可以上台...",-2,-2,-2,-2,1,-2,-2,0,...,-2,-2,-2,-2,1,1,-2,-2,1,1
8,8,"""超喜欢这家的面食，经常来吃。这家风格及产品都很像京城御面，有时会怀疑是不是同一家的。面条比...",-2,-2,-2,-2,-1,-2,-2,-2,...,1,-2,-2,-2,-1,0,-2,-2,1,-2
9,9,"""广西阳朔，前一天晚上吃的特色啤酒鱼，就看到了隔壁有家螺蛳粉，第二天过来吃。个人比较喜欢粉，...",-2,-2,-2,-2,1,-2,-2,-1,...,1,-2,-2,-2,1,0,1,-2,1,1


In [5]:
df_testa_pred.shape

(15000, 22)

In [7]:
df_testa_pred[df_testa_pred.columns[2:]][0:3]

,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,price_cost_effective,price_discount,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,1,-2
1,-2,-2,-2,-2,-2,1,-2,0,-2,0,-2,-2,-2,-2,1,1,-2,1,1,1
2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,-2,-2,-2,1,1,0,-2,-2,1,1


## 查看valid set推测结果

In [104]:
df_valid = pd.read_csv('./data/df_valid.csv')
df_valid_pred = pd.read_csv('./data/df_valid_pred.csv')

In [118]:
idx = 2

In [119]:
print(df_valid['content'].iloc[idx])

"这家店是我目前吃到的最干净的串串店，目前看到最有意思的串串店。
菜单是张试题卷，很有新意，和80后餐厅有点像，比较怀旧，装饮料的杯子也是80年代的搪瓷杯！

整体味道还不错，至少锅底看起来挺干净的。不过锅底味道一般，点的鸳鸯锅，基本没吃过白锅，太咸，也不鲜。下次直接红锅就好了。

2个人干掉将近60串串串，感觉还是挺能吃的。
老板服务什么都还不错，小工就差很多，看来还需要多培训培训。
当场点评还能送一杯冰粉，冰粉好大一杯，感觉不需要再点饮料了。

唯一让我很想吐槽的就是油碟——花生酱之类的酱料竟然不能续加！！！不能续加！！！上来给我的一碟本来就不多，吃完了还不让加！！！还要再付钱！！！下次去吃，油碟必须加满！"


In [120]:
pd.concat([df_valid.iloc[idx, 2:], df_valid_pred.iloc[idx, 2:]], axis=1)

,2,2
location_traffic_convenience,-2,-2
location_distance_from_business_district,-2,-2
location_easy_to_find,-2,-2
service_wait_time,-2,-2
service_waiters_attitude,0,1
service_parking_convenience,-2,-2
service_serving_speed,-2,-2
price_level,-2,-2
price_cost_effective,-2,-2
price_discount,-2,-2


In [76]:
# 查看真值和推测值每个粒度下类别unique值
for i in range(20):
    print('{:2}: true: {}, pred: {}'.format(i+1, df_valid.iloc[:, i+2].unique(), df_valid_pred.iloc[:, i+2].unique()))

 1: true: [-2  1 -1  0], pred: [-2  1 -1]
 2: true: [-2  1  0 -1], pred: [-2  1]
 3: true: [-2 -1  0  1], pred: [-2  1 -1]
 4: true: [ 0 -2 -1  1], pred: [-2]
 5: true: [-2  0  1 -1], pred: [-2  1  0 -1]
 6: true: [-2 -1  1  0], pred: [-2  1  0 -1]
 7: true: [-2  0  1 -1], pred: [-2  1 -1]
 8: true: [ 1 -2  0 -1], pred: [ 1  0 -2 -1]
 9: true: [-2  1  0 -1], pred: [-2  1 -1  0]
10: true: [-2  0 -1  1], pred: [-2  0  1]
11: true: [-2  0  1 -1], pred: [-2  1  0 -1]
12: true: [-2  0  1 -1], pred: [-2  1  0 -1]
13: true: [-2  0 -1  1], pred: [-2  0  1 -1]
14: true: [-2  1 -1  0], pred: [-2  1  0 -1]
15: true: [-2  1  0 -1], pred: [-2  1 -1  0]
16: true: [ 0  1 -2 -1], pred: [ 0  1 -1 -2]
17: true: [-2  1  0 -1], pred: [-2  1  0 -1]
18: true: [-2  1 -1  0], pred: [-2  1 -1  0]
19: true: [ 1  0 -1 -2], pred: [ 0  1 -1]
20: true: [ 0  1 -2 -1], pred: [-2  1 -1  0]


In [85]:
np.sum(df_valid.iloc[:, 2] == -2)

11757

In [87]:
np.sum(df_valid_pred.iloc[:, 2] == -2)

12099

## check df_testa_pred

In [154]:
df_testa_pred = pd.read_csv('./data/df_testa_pred.csv')

In [155]:
df_testa_pred.shape

(15000, 22)

In [162]:
for i in range(20):
    print('{:2}: pred: {}'.format(i+1, df_testa_pred.iloc[:, i+2].unique()))

 1: pred: [-2  1 -1]
 2: pred: [-2  1 -1]
 3: pred: [-2  1 -1  0]
 4: pred: [-2 -1  1  0]
 5: pred: [ 1 -2 -1  0]
 6: pred: [-2  1 -1  0]
 7: pred: [-2  1 -1  0]
 8: pred: [ 0 -2 -1  1]
 9: pred: [ 1  0 -2 -1]
10: pred: [ 0 -2  1]
11: pred: [-2  1  0 -1]
12: pred: [-2  1 -1  0]
13: pred: [-2  1  0 -1]
14: pred: [ 1 -2  0 -1]
15: pred: [ 1 -1 -2  0]
16: pred: [ 1  0 -2 -1]
17: pred: [-2  1 -1  0]
18: pred: [-2  1 -1  0]
19: pred: [ 1 -1  0]
20: pred: [ 1 -2 -1  0]


In [159]:
idx = 101

In [160]:
print(df_testa_pred['content'].iloc[idx])

"来魔都朋友推荐吃捞王 来到住所附近的静安店 环境很清幽 中午11点饭点来的 人很少 点了招牌的太极猪肚鸡中份 皇上皇腊味煲仔饭 手工丸类组合 （虾丸 撒尿牛丸 还有花枝丸）精选上脑牛肉还有田园大丰收（蔬菜拼盘）还有些豆腐类乳制品 酱料可以自选 俩人消费量略多 推荐3-4人 汤真的是非常鲜美醇厚 配上赠送的酱料 猪肚和鸡肉都非常美味 不会很腥 服务也是相当周到 招待员霸王花从开始消费到结束都非常热情周到 会帮助我们合理搭配添加的菜品 及时捞出汤沫 烫菜 盛煲仔饭 32个赞 用大众闪惠支付还赠送了一扎活动马蹄水"


In [161]:
pd.DataFrame(df_testa_pred.iloc[idx, 2:])

,101
location_traffic_convenience,-2
location_distance_from_business_district,-2
location_easy_to_find,-2
service_wait_time,-2
service_waiters_attitude,1
service_parking_convenience,-2
service_serving_speed,-2
price_level,-2
price_cost_effective,-2
price_discount,0
